# 1. Data Extraction

## 1-1 Install and import related libraries

In [1]:
# !pip install kaggle
# !pip install kagglehub


import kagglehub
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')


## 1-2 Import dataset and assign to df

In [2]:


# Initialize API
api = KaggleApi()
api.authenticate()

# Create a download directory
download_dir = "laptop_data"
os.makedirs(download_dir, exist_ok=True)

# Download the dataset
api.dataset_download_files("sumanbera19/laptop-price-dataset", path=download_dir, unzip=True)

# Look for the CSV file
for file in os.listdir(download_dir):
    if file.endswith(".csv"):
        csv_path = os.path.join(download_dir, file)
        break

# Load it into a DataFrame
df = pd.read_csv(csv_path)

# Show the first few rows
df.head()



Dataset URL: https://www.kaggle.com/datasets/sumanbera19/laptop-price-dataset


,Unnamed: 0,Model,Price,Rating,Generation,Core,Ram,SSD,Display,Graphics,OS,Warranty
0,0,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,"₹50,399",70.0,5th Gen AMD Ryzen 5 5600H,"Hexa Core, 12 Threads",8 GB DDR4 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels, Touch Screen",4 GB AMD Radeon RX 6500M,Windows 11 OS,1 Year Warranty
1,1,Tecno Megabook T1 Laptop (11th Gen Core i3/ 8G...,"₹23,990",56.0,11th Gen Intel Core i3 1115G4,"Dual Core, 4 Threads",8 GB LPDDR4 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels",Intel UHD Graphics,Windows 11 OS,1 Year Warranty
2,2,Lenovo V15 G4 ‎82YU00W7IN Laptop (AMD Ryzen 3 ...,"₹26,989",55.0,7th Gen AMD Ryzen 3 7320U,"Quad Core, 8 Threads",8 GB LPDDR5 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels",AMD Radeon Graphics,Windows 11 OS,1 Year Warranty
3,3,Samsung Galaxy Book2 Pro 13 Laptop (12th Gen C...,"₹69,990",60.0,12th Gen Intel Core i5 1240P,"12 Cores (4P + 8E), 16 Threads",16 GB LPDDR5 RAM,512 GB SSD,"13.3 inches, 1080 x 1920 pixels",Intel Iris Xe Graphics,Windows 11 OS,1 Year Warranty
4,4,Xiaomi Redmi G Pro 2024 Gaming Laptop (14th Ge...,"₹1,02,990",78.0,14th Gen Intel Core i9 14900HX,"24 Cores (8P + 16E), 32 Threads",16 GB DDR5 RAM,1 TB SSD,"16.1 inches, 2560 x 1600 pixels",8 GB NVIDIA GeForce RTX 4060,Windows 11 OS,1 Year Warranty


In [3]:
df = df.drop(['Unnamed: 0'], axis = 1) # drop Unnamed column as it is not required for ETL process

# 2. Data Transformation
- Goal : Prepare data for data analysis and easy to understand.
- Challenge : Many variables contains a lot of data and data types are string.
- solutions : split a variable to separate variables.


## 2-1 Duplicate and missing values

In [4]:
# check missin values

for column in df.columns:
    print(column)
    print(df[column].isnull().sum())
    print(" ")
print(f'Number of rows { df.shape[0]} in dataset')

Model
0
 
Price
0
 
Rating
152
 
Generation
0
 
Core
0
 
Ram
0
 
SSD
0
 
Display
1
 
Graphics
0
 
OS
1
 
Warranty
5
 
Number of rows 920 in dataset


In [5]:
# found some missing values in Rating, Display, OS, Warranty. number of missing values are small, so eliminate except for Rating.
# update Rating with mean or median of Rating

df['Rating'].describe()

# mean and median are close in small range(50 - 79) . will use mean value
rating_mean = df['Rating'].mean()
df['Rating'].fillna(rating_mean, inplace = True)

In [6]:
df.info()
# Found wron data entered
df[df['Ram'] == 'Storage: 512 GB NVMe SSD']

# Solution 1 - manipulate data -> can be manipulated.
# Solution 2 - eliminate the data -> will not effect much on the result as just one entry
# will go with solution 1

def switch_values(df, row) :
    '''
    core - Ram
    Ram - SSD
    SSD - Graphic
    Display - OS
    Graphics - Display
    OS - Warranty
    '''
    ram = df.loc[row , 'Core']
    ssd = df.loc[row, 'Ram']
    graphic = df.loc[row, 'SSD']
    os = df.loc[row, 'Display']
    display = df.loc[row, 'Graphics']
    warranty = df.loc[row, 'OS']
    
    
    df.loc[row,'Ram'] = ram
    df.loc[row,'SSD'] = ssd
    df.loc[row,'Graphics'] = graphic
    df.loc[row,'OS'] = os
    df.loc[row, 'Display'] = display
    df.loc[row,'Warranty'] = warranty

switch_list = [65,111,180]
for col in switch_list :
    switch_values(df,col)
    print(df.loc[col, :])

ssd = df.loc[108,'Ram']
os = df.loc[108,'SSD']
graphic = df.loc[108,'OS']

df.loc[108,'SSD'] = ssd
df.loc[108,'OS'] = os
df.loc[108,'Graphics'] = graphic


    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Model       920 non-null    object 
 1   Price       920 non-null    object 
 2   Rating      920 non-null    float64
 3   Generation  920 non-null    object 
 4   Core        920 non-null    object 
 5   Ram         920 non-null    object 
 6   SSD         920 non-null    object 
 7   Display     919 non-null    object 
 8   Graphics    920 non-null    object 
 9   OS          919 non-null    object 
 10  Warranty    915 non-null    object 
dtypes: float64(1), object(10)
memory usage: 79.2+ KB
Model         MSI Titan 18 HX Gaming Laptop (14th Gen Core i...
Price                                                 ₹5,99,990
Rating                                                 61.75651
Generation                       14th Gen Intel Core i9 14900HX
Core                                             32 

In [7]:
# Drop rows with missing values of 'Display', 'OS', 'Warranty'
df.dropna(axis = 0, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 915 entries, 0 to 919
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Model       915 non-null    object 
 1   Price       915 non-null    object 
 2   Rating      915 non-null    float64
 3   Generation  915 non-null    object 
 4   Core        915 non-null    object 
 5   Ram         915 non-null    object 
 6   SSD         915 non-null    object 
 7   Display     915 non-null    object 
 8   Graphics    915 non-null    object 
 9   OS          915 non-null    object 
 10  Warranty    915 non-null    object 
dtypes: float64(1), object(10)
memory usage: 85.8+ KB


In [8]:
# Data is free of missing values.
# check duplication

print("Number of duplication in Data : " , df.duplicated().sum())

df = df.reset_index()


Number of duplication in Data :  0


In [9]:
# there is no duplicatd data and missing values.

## 2-2 Data consistency

## 2-3 Data types
1. Model - separate brand names - possibly find space and separate
2. Price - data type to float, data currency to CAD
3. Rating - may be int
4. generation = separate to gen, brand, products
5. core - separe to corres, treads
6. Ram - separate GB, types(LPDDR5, DDR5)
7. SSD - volumns and type(SSD HDD or else)
8. Display - touch, inches, definition
9. Graphic -  volumn, brand, model
10. OS - types(windows 10 11 or mac or linux)
11. Warranty - years covered

In [10]:

# df['Model'].unique()
# many unique values of model name. I can extract brand of the model. 
# the model name is spaced, so i will use it to extract brand name
# build a def to extract data from text

def extract_data(df, column, loc):
    '''
    Extract data from a column by spacing and return data list.
    Argument
        df (data frame) data table to explore
        column (str) name of column to find in df
        loc (int) n_th loc of splited value
    Return
        new_col (list) built list from value extract from df
    '''
    new_col = []
    for i in range(len(df[column])):
        new_col.append(df[column][i].split(" ")[loc])
    return new_col



In [11]:

# df['Generation'].unique()

# generation, brand, model, model number
# generation - ryzen 5,6,7,8 intel 11,12,13, Apple none, Mediatek , qualcom
# brand - AMD, intel, apple, mediatek, qualcom
# model - i series, celeron, ultra, atom, ryzen, athlon, m series, kompanio 
# model number - last split -done
# remove \u200b \u200e - done
# make a change values of long text to formatted text , Processor: 12th Generation Intel Core i5-1235U processor (1.3 GHz up to 4.4 GHz 12 MB L3…



In [12]:
# Create Brand variable to store Brand of product
brand = extract_data(df,'Model',0)
df['Brand'] = brand

# data ingesting of price, eliminate comma and currency sign before change data type to float
df['Price'] = df['Price'].str.replace(",","")
df['Price'] = df['Price'].str.replace("₹","")
df['Price'] = df['Price'].astype('float')

# searched conversion rate from IND to CAD. the Valuse is in CAD
conversion_rate = 0.016
df['Price_CAD'] = round(df['Price']*conversion_rate,2)



In [13]:
# extract model number
def extract_model_number (df, col) :
    '''
    return a list of model number, extract the last split of col as the model number is in last part of text and build a list to return
    Argument
        df (data frame) date table to extract model number from
        col (text) a column name of data frame
    Return
        list (list) a list of model number extract from df with col
    '''
    list = []
    for i in range(len(df[col])):
        last=len(df.loc[i,col].split())-1
        list.append(df.loc[i,col].split()[last])
    return list


In [14]:
# Build a def to achieve efficienty and resuability
import re

def extract_pattern (df, patterns, col) :
    pattern_list = []
    for text in df[col]:
        dummy = 'None'
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                dummy = match.group()
                break  # stop after first match
        pattern_list.append(dummy)
    return pattern_list



In [15]:
# Pattern list
cpu_brand_patterns = [
    'Intel', 'AMD', 'Media*','Apple','Qualcom'
]

model_patterns = [
    'Ryzen \d', 'i\d', 'Ultra \d', 'M\d',
    'Athlon', 'Celeron', 'Atom Quad Core',
    'X Elite', 'Kompanio', 'Pentium'
]

gen_patterns = [
    '\d+th',
    '\d+st',
    '\d+nd',
    '\d+rd'
]


In [16]:
# remove \u200 text , long to short form,
def make_change(df_gen,col) :
    '''
    Make small changes to ensure data consistency of columns of datatable
    1. add NA to rows if there are no model numbers
    2. replace a long value or unformatted value to formatted value
    3. eliminate \u200e and \u200b from value
    '''

    for i in range(len(df_gen[col])) :
        if df_gen.loc[i,col] == '13th Gen Intel Core i7':
            df_gen.loc[i, col] = '13th Gen Intel Core i7 NA'
        elif len(df_gen.loc[i, col].split(' ')) > 8 :
            df_gen.loc[i, col] = '12th Gen Intel Core i5 1235U'
        elif df_gen.loc[i,col] ==  '12th Generation Intel Core i5-1235U' :
            df_gen.loc[i, col] = '12th Gen Intel Core i5 1235U'
        elif df_gen.loc[i,col] ==  'Qualcomm X Elite' :
            df_gen.loc[i, col] = 'Qualcomm X Elite NA'    
        df_gen.loc[i, col] = df_gen.loc[i, col].replace('\u200e','')
        df_gen.loc[i, col] = df_gen.loc[i, col].replace('\u200b','')

In [17]:
cpu = pd.DataFrame(df['Generation'])

make_change(cpu,'Generation')
cpu.head(20)

,Generation
0,5th Gen AMD Ryzen 5 5600H
1,11th Gen Intel Core i3 1115G4
2,7th Gen AMD Ryzen 3 7320U
3,12th Gen Intel Core i5 1240P
4,14th Gen Intel Core i9 14900HX
5,12th Gen Intel Core i3 1215U
6,Apple M1
7,12th Gen Intel Core i5 1235U
8,12th Gen Intel Core i5 1235U
9,7th Gen Amd Ryzen 5 7535HS


In [18]:
cpu['brand'] = extract_pattern(cpu, cpu_brand_patterns,'Generation')
cpu['gen'] = extract_pattern(cpu, gen_patterns,'Generation')
cpu['model'] = extract_pattern(cpu, model_patterns,'Generation')
cpu['Model_Number'] = extract_model_number(cpu,'Generation')

cpu.head(20)

,Generation,brand,gen,model,Model_Number
0,5th Gen AMD Ryzen 5 5600H,AMD,5th,Ryzen 5,5600H
1,11th Gen Intel Core i3 1115G4,Intel,11th,i3,1115G4
2,7th Gen AMD Ryzen 3 7320U,AMD,7th,Ryzen 3,7320U
3,12th Gen Intel Core i5 1240P,Intel,12th,i5,1240P
4,14th Gen Intel Core i9 14900HX,Intel,14th,i9,14900HX
5,12th Gen Intel Core i3 1215U,Intel,12th,i3,1215U
6,Apple M1,Apple,None,M1,M1
7,12th Gen Intel Core i5 1235U,Intel,12th,i5,1235U
8,12th Gen Intel Core i5 1235U,Intel,12th,i5,1235U
9,7th Gen Amd Ryzen 5 7535HS,Amd,7th,Ryzen 5,7535HS


In [19]:
def count_value (df) :
    for col in df.columns :
        print(f'Value counts of {col}')
        print(df[col].value_counts())
        print(" ")

# checking value counts
count_value(cpu)


Value counts of Generation
Generation
12th Gen Intel Core i5 1235U         49
12th Gen Intel Core i3 1215U         45
12th Gen Intel Core i5 12450H        40
13th Gen Intel Core i5 1335U         40
11th Gen Intel Core i3 1115G4        26
                                     ..
13th Gen Intel Core i7 13700HX        1
Intel Pentium Silver N6000            1
7th Gen AMD Ryzen 7040 Series 040     1
13th Gen Intel Core i9 13900HX        1
12th Gen Intel Core i5 1230U          1
Name: count, Length: 168, dtype: int64
 
Value counts of brand
brand
Intel      670
AMD        207
Apple       18
Amd         10
Media        5
intel        4
Qualcom      1
Name: count, dtype: int64
 
Value counts of gen
gen
13th    246
12th    219
7th     118
None    108
11th    103
5th      67
6th      15
14th     11
3rd       9
10th      9
8th       5
4th       5
Name: count, dtype: int64
 
Value counts of model
model
i5                282
i7                156
i3                105
Ryzen 5            99
Ryzen 7 

In [20]:
cpu.to_csv('cpu.csv')

In [21]:
cpu['Generation'].head()

0         5th Gen AMD Ryzen 5 5600H
1     11th Gen Intel Core i3 1115G4
2         7th Gen AMD Ryzen 3 7320U
3      12th Gen Intel Core i5 1240P
4    14th Gen Intel Core i9 14900HX
Name: Generation, dtype: object

In [22]:
df['Core'].unique()

array(['Hexa Core, 12 Threads', 'Dual Core, 4 Threads',
       'Quad Core, 8 Threads', '12 Cores (4P + 8E), 16 Threads',
       '24 Cores (8P + 16E), 32 Threads',
       'Hexa Core (2P + 4E), 8 Threads', 'Octa Core (4P + 4E)',
       '10 Cores (2P + 8E), 12 Threads', 'Octa Core, 16 Threads',
       '12 Cores (2P + 8E + 2LP-E), 14 Threads',
       '16 Cores (6P + 8E + 2LP-E), 22 Threads',
       'Octa Core (4P + 4E), 12 Threads',
       '10 Cores (6P + 4E), 16 Threads', 'Dual Core, 2 Threads',
       '14 Cores (6P + 8E), 20 Threads',
       '14 Cores (4P + 8E + 2LP-E), 18 Threads',
       '20 Cores (8P + 12E), 28 Threads', '32\u2009GB DDR5 RAM',
       '16 Cores (12P + 4E)', '14 Cores (6P + 8E)',
       'Quad Core, 4 Threads', '16 Cores (8P + 8E), 24 Threads',
       'Memory: 8 GB LPDDR4 Memory (On BD 8 GB)', 'Quad Core',
       '32\u2009GB LPDDR5X RAM', '14 Cores (10P + 4E)',
       '11 Cores (5P + 6E)', '14 Cores, 20 Threads',
       '16\u2009GB DDR5 RAM', '5 Cores (1P + 4E), 6 Thread

In [23]:
core = df['Core'].reset_index()

In [24]:
core.head()

,index,Core
0,0,"Hexa Core, 12 Threads"
1,1,"Dual Core, 4 Threads"
2,2,"Quad Core, 8 Threads"
3,3,"12 Cores (4P + 8E), 16 Threads"
4,4,"24 Cores (8P + 16E), 32 Threads"


In [25]:
# find and replace
# 1. hexa-> 6, Quad -> 4, Octa -> 8, dual -> 2
def replace_cores(df) :
    rpc_core_list = []
    for i in range(len(df['Core'])) :
        core_str = df['Core'][i].split()[0]
        dummy_replaced = df['Core'][i]
        if core_str == 'Hexa' :
            dummy_replaced = re.sub('Hexa','6', dummy_replaced)
    
        elif core_str == 'Octa' :
            dummy_replaced = re.sub('Octa','8', dummy_replaced)
            
        elif core_str == 'Quad' :
            dummy_replaced = re.sub('Quad','4', dummy_replaced)
    
        elif core_str == 'Dual' :
            dummy_replaced = re.sub('Dual', '2', dummy_replaced)
        rpc_core_list.append(dummy_replaced)

    return rpc_core_list

core['Core_replaced'] = replace_cores(core)

In [26]:

def extract_core(df) :
    core_list = []
    for i in range(len(df['Core_replaced'])) :
        dummy = 'None'
        text = df['Core_replaced'][i].split(',')[0]
        match = re.search('.+Core.*', text, re.IGNORECASE)
        if match :
            dummy = match.group()
        core_list.append(dummy) 
    
    return core_list

cores = extract_core(core)
core['cores'] = cores    
core.head()

,index,Core,Core_replaced,cores
0,0,"Hexa Core, 12 Threads","6 Core, 12 Threads",6 Core
1,1,"Dual Core, 4 Threads","2 Core, 4 Threads",2 Core
2,2,"Quad Core, 8 Threads","4 Core, 8 Threads",4 Core
3,3,"12 Cores (4P + 8E), 16 Threads","12 Cores (4P + 8E), 16 Threads",12 Cores (4P + 8E)
4,4,"24 Cores (8P + 16E), 32 Threads","24 Cores (8P + 16E), 32 Threads",24 Cores (8P + 16E)


In [27]:
text = core['Core_replaced'][2].split(',')
len(text)

2

In [28]:
# split cores and treads and other
# split cores by , treads after , , None,
def extract_thread (df) :
    tread_list = []
    for i in range(len(df['Core_replaced'])) :
        dummy = 'None'
        text = df['Core_replaced'][i].split(',')
        if len(text) < 2 :
            loc = 0
        else :
            loc = 1
        match = re.search('.\d*.Threads',text[loc] , re.IGNORECASE)
        if match :
            dummy = match.group()
        tread_list.append(dummy)

    return tread_list
core_thread = extract_thread(core)
core['thread'] = core_thread


In [29]:
core[core['thread'] == 'None']
core_final = core[['Core_replaced','cores','thread']]

core_final.to_csv('core.csv')

In [30]:
core_final.head()

,Core_replaced,cores,thread
0,"6 Core, 12 Threads",6 Core,12 Threads
1,"2 Core, 4 Threads",2 Core,4 Threads
2,"4 Core, 8 Threads",4 Core,8 Threads
3,"12 Cores (4P + 8E), 16 Threads",12 Cores (4P + 8E),16 Threads
4,"24 Cores (8P + 16E), 32 Threads",24 Cores (8P + 16E),32 Threads


In [31]:
df.head()

,index,Model,Price,Rating,Generation,Core,Ram,SSD,Display,Graphics,OS,Warranty,Brand,Price_CAD
0,0,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,50399.0,70.0,5th Gen AMD Ryzen 5 5600H,"Hexa Core, 12 Threads",8 GB DDR4 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels, Touch Screen",4 GB AMD Radeon RX 6500M,Windows 11 OS,1 Year Warranty,HP,806.38
1,1,Tecno Megabook T1 Laptop (11th Gen Core i3/ 8G...,23990.0,56.0,11th Gen Intel Core i3 1115G4,"Dual Core, 4 Threads",8 GB LPDDR4 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels",Intel UHD Graphics,Windows 11 OS,1 Year Warranty,Tecno,383.84
2,2,Lenovo V15 G4 ‎82YU00W7IN Laptop (AMD Ryzen 3 ...,26989.0,55.0,7th Gen AMD Ryzen 3 7320U,"Quad Core, 8 Threads",8 GB LPDDR5 RAM,512 GB SSD,"15.6 inches, 1920 x 1080 pixels",AMD Radeon Graphics,Windows 11 OS,1 Year Warranty,Lenovo,431.82
3,3,Samsung Galaxy Book2 Pro 13 Laptop (12th Gen C...,69990.0,60.0,12th Gen Intel Core i5 1240P,"12 Cores (4P + 8E), 16 Threads",16 GB LPDDR5 RAM,512 GB SSD,"13.3 inches, 1080 x 1920 pixels",Intel Iris Xe Graphics,Windows 11 OS,1 Year Warranty,Samsung,1119.84
4,4,Xiaomi Redmi G Pro 2024 Gaming Laptop (14th Ge...,102990.0,78.0,14th Gen Intel Core i9 14900HX,"24 Cores (8P + 16E), 32 Threads",16 GB DDR5 RAM,1 TB SSD,"16.1 inches, 2560 x 1600 pixels",8 GB NVIDIA GeForce RTX 4060,Windows 11 OS,1 Year Warranty,Xiaomi,1647.84


In [32]:
# remove '\u2009' in the values
for i in range(len(df['Ram'])) :
        
    df['Ram'][i] = df['Ram'][i].replace('\u2009','')

for i in range(len(df['Ram'])) :
    loc = len(df['Ram'][1].split(' ')) - 1


In [33]:
#split memory and generation
ram = df['Ram'].reset_index()

ram['Ram'][0]
re.search('\dGB',ram['Ram'][0]).group()

ram_pattern = ['\d\dGB','\dGB']
ram_gb = extract_pattern(ram, ram_pattern,'Ram')

ram_memory_pattern = ['LPDDR\d\D', 'LPDDR\d','DDR\d']
ram_memory = extract_pattern(ram, ram_memory_pattern,'Ram')
ram['memory_gb'] = ram_gb
ram['memory_type'] = ram_memory

ram['memory_type'].unique()

ram.drop(['index'], axis = 1, inplace = True)
ram.to_csv('ram.csv')

In [34]:


for i in range(len(df['SSD'])) :
        
    df['SSD'][i] = df['SSD'][i].replace('\u2009','')

for i in range(len(df['SSD'])) :
    loc = len(df['SSD'][1].split(' ')) - 1

In [35]:
ssd = df['SSD'].reset_index()

In [36]:
ssd['SSD'].unique()
ssd_pattern = ['NVMe SSD' , 'SSD','Hard Disk'] 
ssd_disk = extract_pattern(ssd, ssd_pattern, 'SSD')
ssd_disk
ssd_size_pattern = ['\d*.GB',
                    '\d*GB',
                   '\d*TB',
                    ]

ssd_disk_size = extract_pattern(ssd, ssd_size_pattern, 'SSD')

In [37]:
ssd['ssd_size'] = ssd_disk_size
ssd['ssd_type'] = ssd_disk
ssd['ssd_size'].unique()

array(['512GB', '1TB', '256GB', '2TB', '128GB', '8GB', 'None', '32GB',
       '4TB', '64GB'], dtype=object)

In [38]:
ssd.drop(['index'],axis = 1, inplace = True)
ssd.to_csv('ssd.csv')


## 

In [39]:
df.head()
df['Display'] = df['Display'].str.replace('\u2009',' ')
df['Display'] = df['Display'].str.replace('\u200e',' ')

In [40]:
'4 GB  NVIDIA Quadro T600','4 GB NVIDIA GeForce RTX 3050 ','4 GB NVIDIA GeForce RTX 2050 '
swab_list = [83,326,594,595,767]

def swab (df,row) :
    hd = df.loc[row, 'SSD']
    gra = df.loc[row, 'Display']
    ssd = df.loc[row, 'Graphics']
    display = df.loc[row, 'OS']
    os = df.loc[row, 'Warranty']

    df.loc[row,'SSD'] = hd + ',' + ssd
    df.loc[row, 'Graphics'] = gra
    df.loc[row, 'Display'] = display
    df.loc[row, 'OS'] = os

for row in swab_list :
    swab(df,row)
  

In [41]:
df['Display'].unique()


array(['15.6 inches, 1920 x 1080 pixels, Touch Screen',
       '15.6 inches, 1920 x 1080 pixels',
       '13.3 inches, 1080 x 1920 pixels',
       '16.1 inches, 2560 x 1600 pixels',
       '13.3 inches, 2560 x 1600 pixels',
       '15.6 inches, 1080 x 1920 pixels',
       '16.1 inches, 1920 x 1080 pixels',
       '13.6 inches, 2560 x 1664 pixels', '14 inches, 1920 x 1080 pixels',
       '14 inches, 1920 x 1200 pixels',
       '14 inches, 1920 x 1200 pixels, Touch Screen',
       '14 inches, 2880 x 1800 pixels',
       '13.4 inches, 1920 x 1200 pixels, Touch Screen',
       '16 inches, 2880 x 1800 pixels, Touch Screen',
       '16 inches, 1920 x 1200 pixels', '15.6 inches, 1366 x 768 pixels',
       '16.1 inches, 2560 x 1440 pixels',
       '13.3 inches, 1080 x 1920 pixels, Touch Screen',
       '16 inches, 2560 x 1600 pixels',
       '13.5 inches, 3000 x 2000 pixels, Touch Screen',
       '16 inches, 1440 x 2560 pixels', '18 inches, 3840 x 2400 pixels',
       '15.6 inches, 2560 x 1440

In [42]:
df[df['Display'] == '4 GB  NVIDIA Quadro T600']

,index,Model,Price,Rating,Generation,Core,Ram,SSD,Display,Graphics,OS,Warranty,Brand,Price_CAD


In [43]:
df[df['Display'] == '4 GB NVIDIA GeForce RTX 3050 ']

,index,Model,Price,Rating,Generation,Core,Ram,SSD,Display,Graphics,OS,Warranty,Brand,Price_CAD


In [44]:
df[df['Display'] == '4 GB NVIDIA GeForce RTX 2050 ']

,index,Model,Price,Rating,Generation,Core,Ram,SSD,Display,Graphics,OS,Warranty,Brand,Price_CAD


In [45]:
display = df['Display'].reset_index()

In [46]:
display['Display'].str.split(',')

0      [15.6 inches,  1920 x 1080 pixels,  Touch Screen]
1                     [15.6 inches,  1920 x 1080 pixels]
2                     [15.6 inches,  1920 x 1080 pixels]
3                     [13.3 inches,  1080 x 1920 pixels]
4                     [16.1 inches,  2560 x 1600 pixels]
                             ...                        
910                     [16 inches,  1920 x 1200 pixels]
911                   [15.6 inches,  2560 x 1440 pixels]
912                     [14 inches,  1920 x 1080 pixels]
913                   [15.6 inches,  1920 x 1080 pixels]
914                   [15.6 inches,  1920 x 1080 pixels]
Name: Display, Length: 915, dtype: object

In [47]:
display['Display'].unique()

array(['15.6 inches, 1920 x 1080 pixels, Touch Screen',
       '15.6 inches, 1920 x 1080 pixels',
       '13.3 inches, 1080 x 1920 pixels',
       '16.1 inches, 2560 x 1600 pixels',
       '13.3 inches, 2560 x 1600 pixels',
       '15.6 inches, 1080 x 1920 pixels',
       '16.1 inches, 1920 x 1080 pixels',
       '13.6 inches, 2560 x 1664 pixels', '14 inches, 1920 x 1080 pixels',
       '14 inches, 1920 x 1200 pixels',
       '14 inches, 1920 x 1200 pixels, Touch Screen',
       '14 inches, 2880 x 1800 pixels',
       '13.4 inches, 1920 x 1200 pixels, Touch Screen',
       '16 inches, 2880 x 1800 pixels, Touch Screen',
       '16 inches, 1920 x 1200 pixels', '15.6 inches, 1366 x 768 pixels',
       '16.1 inches, 2560 x 1440 pixels',
       '13.3 inches, 1080 x 1920 pixels, Touch Screen',
       '16 inches, 2560 x 1600 pixels',
       '13.5 inches, 3000 x 2000 pixels, Touch Screen',
       '16 inches, 1440 x 2560 pixels', '18 inches, 3840 x 2400 pixels',
       '15.6 inches, 2560 x 1440